In [1]:
import GRASPy as gp 
import pandas as pd

Documentation can be found here - https://sebporras.github.io/GRASPy/



# Performing joint reconstructions.

Step 1) Submit the job to the server

In [2]:
request = gp.JointReconstruction(aln="./test_data/joint_recon/GRASPTutorial_Final.aln",
                                nwk="./test_data/joint_recon/GRASPTutorial_Final.nwk", 
                                alphabet="Protein")

job_id = request["Job"]

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

{'Message': 'Queued', 'Job': 83}


Step 2) Find out where your job is in the queue or the status

In [4]:
#queue = g_requests.PlaceInQueue(job_id)
status = gp.JobStatus(job_id)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

{'Status': 'COMPLETED', 'Job': 49}


Step 3) Retrieve your job which will have the POG graphs 

In [3]:
graphs = gp.JobOutput(job_id)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



Step 4 - Optional) Request POGs for extant sequences 

In [5]:
extant_tree = gp.ExtantPOGTree(aln="./test_data/big_test_data/GRASPTutorial_Final.aln",
nwk="./test_data/big_test_data/GRASPTutorial_Final.nwk")


Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



In [ ]:
extant_tree

Step 5 - Option 1) Build a POG tree from extant and ancesor POGs using both of the server outputs 

- The advantage of doing it this way is that the POGTree object will contain sequence information on the BranchPoints for ancestors AND extants 

In [7]:
tree = gp.POGTreeFromJSON(extant_tree, graphs)

tree.branchpoints["XP_012687241.1"].seq

'PPGPTPWPLVGNFGGFLVPSFISRRFGRNREEYAQRTSNPVSPQVGLMELSKIYGNIYSIFVGSQLMVLLTGYEVVRDALSNHAEVFSDRPDVPIITIMTKRKGIVFAPYGPVWRKQRKFCHSTLRSFGLGKLSLEPCILEGVSIVKSELLSLSNESGEMGVDPTPLISNAVSNVISSISLGQRFHHQDQEFRTLLDLMARGLEISVNSHAVLINIFPWLYHMPCGVFKELRQVEGDITAFLKRIIARHRATLDPENPRDLIDMYLVEMLAQQTAAEPTENTFSEDYLFYIIGDLFIAGTDTTTNSMLWILLYMCLYPDIQEKVQEEIDAVIGRDKVPSLTDKGCLSFTEATIMEVQRMTVVVPLAIPHMASKTTVFRGYTIPKGTVIIPNLWSVHRDPSVWDSPDDFNPSRFLDEQGKLLRKDYFIPFGIGRRVCMGEQLAKMELFLMFTNLMQAFTFRFPKGKPPPPMHGRFGLTLAPCPYTVCVTPR'

Step 5 - Option 2) Build a POGTree from the ancestor POG and from a nwk file string 

- ONLY ancestors will have sequence information based on the most likely symbol at each position in the sequence

In [4]:
tree = gp.POGTreeFromJSON(nwk="./test_data/joint_recon/GRASPTutorial_Final.nwk", POG_graphs=graphs)

In [5]:
tree.writeToNwk(file_name="test_nwk")

'(((XP_006629927.2:0.14777721590070114,(XP_018611667.1:0.31741219299255685,((XP_007229530.1:0.20410994427385432,(XP_012687241.1:0.12553661262814741,XP_018919739.1:0.17752015624894124)N6:0.02108546966591862)N5:0.024498458258308764,(XP_019717376.1:0.3374131974537633,(XP_010886716.1:0.17046994270732152,(XP_014050304.1:0.012222797694125376,XP_021429054.1:0.016443894223391986)N9:0.051214958442506786)N8:0.06898824252200719)N7:0.07186524457989396)N4:0.10310490468352707)N3:0.06898460428141462)N2:0.11632291055872579,(ARO89866.1:0.25205280271933694,(XP_014733783.1:0.29681349725859363,(XP_005082857.1:0.1589210754372825,((XP_021540185.1:0.03325108724429082,XP_019684690.2:0.018088332404641694)N14:0.04823406553019316,(XP_012621711.1:0.05681896074666115,((NP_898898.1:1.0000005001842283e-06,XP_526649.2:0.00220424589188406)N17:0.005288422325157249,(XP_012291909.1:0.0046099502113805535,XP_003929520.1:0.004241068270090409)N18:0.010508512073819531)N16:0.040465306764365216)N15:0.031854968122094096)N13:0.03

# Learning distributions from data 

The following instructions demonstrate how to learn a probability distribution from data. I need to add option to change some of the parameters as currently just runs on default settings. 

Step 1 - Optional) The data below is a demonstration of how data should be formated in a CSV.

- Currently only implemented to accept 1 data point per sequence

In [ ]:
df = pd.DataFrame()

headers = ["A5ILB0","P08144","P29957","H2N0D4","T1WDH2","T1WE96","H9B4I9","A0A060DAC6","Q47R94","Q5UZY3","D8J7H2","Q4A3E0","L8B068","Q2QC88","O93647","Q4J9M2","Q97YY0","O33476","D0KTV8","P96107","P06279","P04746","P00690","O08452","D4P4Y7","Q1KLC8","Q8A1G3","P17654","B8Y698","P00692","B8Y1H0","B1VK33","B5ARZ9","P20845","B6RB08","P06278","P04063","P00693","Q8LJQ6","Q2KJQ1","Q6WUB6","A8VWC5","A0SEG1","P04745","Q75UG5"]
data = [8.5,7.35,7.35,7,9,9,None,8.5,7.35,None,None,None,7,5.75,5.25,3.25,None,6.5,None,None,None,None,7.35,5.75,5.5,7,None,None,8,6,None,7,6.5,None,6.5,6.466666666666666,None,None,None,None,9.5,None,None,6.9,7]

df["Headers"] = headers
df["Data"] = data

df.to_csv('train_data.csv', index=False)


Step 2) Send request to the server

In [6]:
import GRASPy as gp 

request_2 = gp.LearnLatentDistributions(nwk="./test_data/EMTrain/3_2_1_1_filt.nwk", 
                                        states=["A", "B"],                                        
                                        data="./test_data/EMTrain/3_2_1_1_data.csv")

second_id = request_2["Job"]

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

{'Message': 'Queued', 'Job': 84}


Step 3) Check the status of your job 

In [2]:
place = gp.PlaceInQueue(second_id)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

{'Job': 81, 'Place': 0}


Step 4) Retrieve your job and save the output

In [7]:
out = gp.JobOutput(second_id)

out

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



{'Job': 84,
 'Result': {'Distrib': {'Condition': [['A'], ['B']],
   'Pr': [[6.578658560392297, 0.5003342721876145],
    [8.632049675842904, 0.5003342721876145]],
   'Index': [0, 1],
   'Domain': 'dat.Continuous@14e6de80'}}}

Step 5) The learnt distribution can then be marginalised on an ancestor node

In [8]:
j_distrib = out["Result"]["Distrib"]

infer = gp.MarginaliseDistOnAncestor(nwk="./test_data/EMTrain/3_2_1_1_filt.nwk", 
                        states=["A", "B"], 
                        data="./test_data/EMTrain/3_2_1_1_data.csv",
                        distrib=j_distrib,
                        ancestor=0)

job_three_id = infer["Job"]

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

{'Message': 'Queued', 'Job': 85}


In [9]:
infered_distribution = gp.JobOutput(job_three_id)

infered_distribution

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



{'Job': 85,
 'Result': {'N0': [{'Pr': [0.787080600445937, 0.21291939955406308],
    'Domain': {'Size': 2, 'Values': ['A', 'B'], 'Datatype': 'String'}}]}}